# Insurance Calculation

### 1. Reading commision sheet

In [3]:
import pandas as pd

# df = pd.read_excel("/home/choice/Desktop/Insuarance/Motor July grid (1).xlsx", sheet_name="Chola CV_Jul 24")

# Chola TW_Jul-24
df = pd.read_excel("/home/choice/Desktop/Insuarance/New_India_June24.xlsx")

In [4]:
df.head(8)

,Motor Insurance :(Private Car & Two Wheeler),Unnamed: 1,Unnamed: 2
0,Product,OD Commission (including Rewards),TP Commission (including Rewards)
1,Private Car,20,15
2,Private Car Standalone OD,20,NaN
3,Private Car Standalone TP,NaN,15
4,Two Wheeler,25,10
5,Two Wheeler Standalone OD,20,NaN
6,Two Wheeler Standalone TP,NaN,10
7,CPA for Pvt Car / TW,NaN,3.25


### 2. Handling Sheet

In [5]:
# Defining from which row, the table/header is starting 
# TODO: Make it dynamic

df_range = 0

In [6]:
df = df.iloc[df_range:]

In [7]:
df.head(5)

,Motor Insurance :(Private Car & Two Wheeler),Unnamed: 1,Unnamed: 2
0,Product,OD Commission (including Rewards),TP Commission (including Rewards)
1,Private Car,20,15
2,Private Car Standalone OD,20,NaN
3,Private Car Standalone TP,NaN,15
4,Two Wheeler,25,10


In [8]:
# Step 2: Drop rows with all NaN values
df = df.dropna(how='all').reset_index(drop=True)

# Drop columns where all values are NaN
df = df.dropna(axis=1, how='all')

In [9]:
df.head(5)

,Motor Insurance :(Private Car & Two Wheeler),Unnamed: 1,Unnamed: 2
0,Product,OD Commission (including Rewards),TP Commission (including Rewards)
1,Private Car,20,15
2,Private Car Standalone OD,20,NaN
3,Private Car Standalone TP,NaN,15
4,Two Wheeler,25,10


In [10]:
# Flash fill for all columns

## As per the above we see that there is null value in the column and that si due to the merge value in the original data to cover this filling the null value with previouse one
df = df.fillna(method='ffill', axis=1)

/tmp/ipykernel_15264/1606804452.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='ffill', axis=1)


In [11]:
df.head(5)

,Motor Insurance :(Private Car & Two Wheeler),Unnamed: 1,Unnamed: 2
0,Product,OD Commission (including Rewards),TP Commission (including Rewards)
1,Private Car,20,15
2,Private Car Standalone OD,20,20
3,Private Car Standalone TP,Private Car Standalone TP,15
4,Two Wheeler,25,10


In [12]:
# Defining which row is the heading

heading_row = 0

In [13]:
df.columns = df.iloc[heading_row]

In [14]:
df.head(5)

,Product,OD Commission (including Rewards),TP Commission (including Rewards)
0,Product,OD Commission (including Rewards),TP Commission (including Rewards)
1,Private Car,20,15
2,Private Car Standalone OD,20,20
3,Private Car Standalone TP,Private Car Standalone TP,15
4,Two Wheeler,25,10


In [15]:
# Dropping index of the heading

df = df.drop(index=heading_row)

In [16]:
df.head(5)

,Product,OD Commission (including Rewards),TP Commission (including Rewards)
1,Private Car,20,15
2,Private Car Standalone OD,20,20
3,Private Car Standalone TP,Private Car Standalone TP,15
4,Two Wheeler,25,10
5,Two Wheeler Standalone OD,20,20


In [17]:
# Step 1: Replace null or unnamed headers with inferred names (if needed)
unnamed_columns = df.columns.str.contains('^Unnamed')
new_columns = []
last_valid_header = None

for i, col in enumerate(df.columns):
    if unnamed_columns[i]:
        if last_valid_header is not None:
            new_columns.append(last_valid_header)
        else:
            new_columns.append('Unnamed')
    else:
        new_columns.append(col)
        last_valid_header = col

df.columns = new_columns

# Function to find the first numerical value row index
def find_first_numerical_row(df):
    for idx, row in df.iterrows():
        if row.apply(lambda x: isinstance(x, (int, float)) or (isinstance(x, str) and x.isdigit())).any():
            return idx
    return None

# Step 2: Find the first numerical row
first_num_row_idx = find_first_numerical_row(df)

# Step 3: Extract original headers and rows above the first numerical row
original_headers = df.columns.tolist()
above_rows = df.iloc[:first_num_row_idx]

# first_row = above_rows.iloc[0].tolist()

# Collect all values from above rows into a single list
above_rows_values = []
for idx in range(first_num_row_idx):
    above_rows_values.extend(above_rows.iloc[idx].tolist())

# # Step 3: Combine the original headers with the first row of data
# combined_headers = [f"{str(header)}_{str(first_row_value)}" for header, first_row_value in zip(original_headers,  first_row)]


combined_headers =  [f"{str(header)}_{str(value)}" for header, value in zip(original_headers, above_rows_values)]
# # Step 4: Create a new DataFrame with combined headers and the rest of the data
# df = pd.DataFrame(df.iloc[1:].values, columns=combined_headers)

new_df = pd.DataFrame(df.iloc[first_num_row_idx:].values, columns=combined_headers)

In [18]:
df.head(5)

,Product,OD Commission (including Rewards),TP Commission (including Rewards)
1,Private Car,20,15
2,Private Car Standalone OD,20,20
3,Private Car Standalone TP,Private Car Standalone TP,15
4,Two Wheeler,25,10
5,Two Wheeler Standalone OD,20,20


In [19]:
## As per the above we see that there is null value in the column and that si due to the merge value in the original data to cover this filling the null value with previouse one

# Flash Fill for columns
new_df = df.fillna(method='ffill', axis=0)

/tmp/ipykernel_15264/2144302754.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  new_df = df.fillna(method='ffill', axis=0)


In [20]:
new_df.head(5)

,Product,OD Commission (including Rewards),TP Commission (including Rewards)
1,Private Car,20,15
2,Private Car Standalone OD,20,20
3,Private Car Standalone TP,Private Car Standalone TP,15
4,Two Wheeler,25,10
5,Two Wheeler Standalone OD,20,20


In [21]:
# Index from which commision starts
index = 1

value = new_df.columns[:index]

In [22]:
value

Index(['Product'], dtype='object')

In [23]:
new_df.columns.dtype

dtype('O')

In [24]:
# Melt the DataFrame
id_vars = value
value_vars = [col for col in new_df.columns if col not in id_vars]
melted_df = pd.melt(new_df, id_vars=id_vars, value_vars=value_vars, 
                    var_name='variable', value_name='value')

In [25]:
melted_df

,Product,variable,value
0,Private Car,OD Commission (including Rewards),20
1,Private Car Standalone OD,OD Commission (including Rewards),20
2,Private Car Standalone TP,OD Commission (including Rewards),Private Car Standalone TP
3,Two Wheeler,OD Commission (including Rewards),25
4,Two Wheeler Standalone OD,OD Commission (including Rewards),20
5,Two Wheeler Standalone TP,OD Commission (including Rewards),Two Wheeler Standalone TP
6,CPA for Pvt Car / TW,OD Commission (including Rewards),CPA for Pvt Car / TW
7,Motor Insurance :(Other than Private Car & Two...,OD Commission (including Rewards),Motor Insurance :(Other than Private Car & Two...
8,YEAR,OD Commission (including Rewards),Motor (Comprehensive)
9,Certificate of registration - 1 to 3 year,OD Commission (including Rewards),15% (OD portion) + Nil -TP portion)


In [26]:
import pandas as pd

# Read the Excel file into a DataFrame
# df = pd.read_excel('your_file.xlsx')

# Sample DataFrame


# Define the hierarchical levels
levels = melted_df.columns

def build_tree(df, levels):
    if len(levels) == 1:
        return df[levels[0]].tolist()
    
    unique_values = df[levels[0]].unique()
    tree = {}
    for value in unique_values:
        subtree_df = df[df[levels[0]] == value]
        tree[value] = build_tree(subtree_df, levels[1:])
    
    return tree

# Build the hierarchical tree
tree = build_tree(melted_df, levels)

# Print the tree
import pprint
pprint.pprint(tree)


{'CPA for Pvt Car / TW': {'OD Commission (including Rewards)': ['CPA for Pvt '
                                                                'Car / TW'],
                          'TP Commission (including Rewards)': ['3.25']},
 'Certificate of registration - 1 to 3 year': {'OD Commission (including Rewards)': ['15% '
                                                                                     '(OD '
                                                                                     'portion) '
                                                                                     '+ '
                                                                                     'Nil '
                                                                                     '-TP '
                                                                                     'portion)'],
                                               'TP Commission (including Rewards)': ['2.5']},
 'Certificate of regist

In [27]:
import json

with open("New India.json", "w") as jsonFile:
    json.dump(tree, jsonFile, indent=4)